In [1]:
import os
import pathlib
import re

import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output, State
import cufflinks as cf

ModuleNotFoundError: No module named 'cufflinks'

In [2]:
DEFAULT_OPACITY = 0.8

mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNrOWJqb2F4djBnMjEzbG50amg0dnJieG4ifQ.Zme1-Uzoi75IaFbieBDl3A"
mapbox_style = "mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz"

base_url = "https://raw.githubusercontent.com/jackparmer/mapbox-counties/master/"

BINS = [
    "0-2",
    "2.1-4",
    "4.1-6",
    "6.1-8",
    "8.1-10",
    "10.1-12",
    "12.1-14",
    "14.1-16",
    "16.1-18",
    "18.1-20",
    "20.1-22",
    "22.1-24",
    "24.1-26",
    "26.1-28",
    "28.1-30",
    ">30",
]

DEFAULT_COLORSCALE = [
    "#f2fffb",
    "#bbffeb",
    "#98ffe0",
    "#79ffd6",
    "#6df0c8",
    "#69e7c0",
    "#59dab2",
    "#45d0a5",
    "#31c194",
    "#2bb489",
    "#25a27b",
    "#1e906d",
    "#188463",
    "#157658",
    "#11684d",
    "#10523e",
]

In [15]:
cm = dict(zip(BINS, DEFAULT_COLORSCALE))
bin = "0-2"

In [16]:
year = 2003
geo_layer = dict(
    sourcetype="geojson",
    source=base_url + str(year) + "/" + bin + ".geojson",
    type="fill",
    color=cm[bin],
    opacity=DEFAULT_OPACITY,
    # CHANGE THIS
    fill=dict(outlinecolor="#afafaf"),
)

In [18]:
lat = figure["layout"]["mapbox"]["center"]["lat"]
lon = figure["layout"]["mapbox"]["center"]["lon"]
zoom = figure["layout"]["mapbox"]["zoom"]

In [ ]:
layout["mapbox"]["layers"].append(geo_layer)

In [6]:
dcc.Graph(
    id="county-choropleth",
    figure=dict(
        layout=dict(
            mapbox=dict(
                layers=[],
                accesstoken=mapbox_access_token,
                style=mapbox_style,
                center=dict(
                    lat=38.72490, lon=-95.61446
                ),
                pitch=0,
                zoom=3.5,
            ),
            autosize=True,
        ),
    ),
)

Graph(id='county-choropleth', figure={'layout': {'mapbox': {'layers': [], 'accesstoken': 'pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNrOWJqb2F4djBnMjEzbG50amg0dnJieG4ifQ.Zme1-Uzoi75IaFbieBDl3A', 'style': 'mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz', 'center': {'lat': 38.7249, 'lon': -95.61446}, 'pitch': 0, 'zoom': 3.5}, 'autosize': True}})

In [9]:
fp = r'C:\Users\rmartinez4\Box\Personal Git\Data-Visualization-DSE241\Final Project\dash_plotly_code\example\data\lat_lon_counties.csv'
df_lat_lon = pd.read_csv(fp)

In [10]:
data = [
    dict(
        lat=df_lat_lon["Latitude "],
        lon=df_lat_lon["Longitude"],
        text=df_lat_lon["Hover"],
        type="scattermapbox",
        hoverinfo="text",
        marker=dict(size=5, color="white", opacity=0),
    )
]

In [17]:
lat = 38.72490
lon = -95.61446
zoom = 3.5

In [19]:
annotations = [
    dict(
        showarrow=False,
        align="right",
        text="<b>Age-adjusted death rate<br>per county per year</b>",
        font=dict(color="#2cfec1"),
        bgcolor="#1f2630",
        x=0.95,
        y=0.95,
    )
]

In [20]:
layout = dict(
    mapbox=dict(
        layers=[],
        accesstoken=mapbox_access_token,
        style=mapbox_style,
        center=dict(lat=lat, lon=lon),
        zoom=zoom,
    ),
    hovermode="closest",
    margin=dict(r=0, l=0, t=0, b=0),
    annotations=annotations,
    dragmode="lasso",
)

In [23]:
test = dcc.Graph(figure = dict(data=data, layout=layout))

In [25]:
test.show()

AttributeError: 'Graph' object has no attribute 'show'